# Middle Lower German Corpus transformation

In [14]:
from lxml import etree
from pathlib import Path
from pathlib import PurePosixPath
from datetime import datetime
import os
import pandas as pd
import re

In [2]:
pd.set_option('display.max_colwidth', 0)

In [11]:
corpus_name = "CorA-ReN-XML_1.1" #only used for export file name
directoryPath = "../data/in/REN/CorA-ReN-XML_1.1/"
fileExtension = ".xml"

In [4]:
def get_cei_files(base_dir):
    for entry in os.scandir(base_dir):
        if entry.is_file() and entry.name.endswith(fileExtension):
            yield Path(entry.path)
        elif entry.is_dir():
            yield from get_cei_files(entry.path)
        else:
            continue

In [5]:
paths = [f"{PurePosixPath(path)}" for path in get_cei_files(directoryPath)]

In [18]:
lists = ["ids", "names", "header", "tokens", "topics", "textTypes", "genres", "languages", "languageTypes", "languageRegions", "languageAreas", "centuries", "dates", "tokens_utf"]
ids, names, header, tokens_ascii, topics, textTypes, genres, languages, languageTypes, languageRegions, languageAreas, centuries, dates, tokens_utf = ([] for i in range(len(lists)))

In [19]:
for file in paths:
    with open(file, "r", encoding="utf-8") as f:
        root = etree.parse(f).getroot()
        ids.append(root.xpath("/text/@id", smart_strings = False))
        names.append(root.xpath("/text/cora-header/@name", smart_strings = False))
        header.append(root.xpath("/text/header/text()", smart_strings = False))
        tokens_utf.append(root.xpath("/text/token/dipl/@utf", smart_strings = False))

In [20]:
contents = list(zip(ids, names, header, tokens_utf))
contents_full = pd.DataFrame(contents).rename(columns={0: "ids", 1: "names", 2: "header", 3: "tokens_utf"})

In [23]:
contents_full["tokens_utf_as_string"] = contents_full["tokens_utf"].str.join(sep=" ")
contents_full["tokens_utf_as_string"] = [re.sub(r' . ', '. ', str(x)) for x in contents_full["tokens_utf_as_string"]]

In [25]:
# name output according to subcorpus

timemarker = datetime.today().strftime('%Y-%m-%d-%H%M')
contents_full.to_json(f"../data/out/{corpus_name}_{timemarker}.json")
contents_full.to_parquet(f"../data/out/{corpus_name}_{timemarker}.parquet")

In [31]:
contents_full

ids                   names  \
0    [5]     [Agneta Willeken]        
1    [M33]   [Alexander Helmst.]      
2    [45]    [Bamberg 1510]           
3    [31]    [Berl. Stb. 1351-1400]   
4    [31]    [Berl. Stb. 1401-1450]   
..    ...                       ...   
230  [T33]   [Wes. Uk. 1351-1400]     
231  [T34]   [Wes. Uk. 1401-1450]     
232  [T35]   [Wes. Uk. 1451-1500]     
233  [M77]   [Westermann 1524]        
234  [M100]  [Wyngaerden Frag. Ms]    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           header  \
0    [text_ReN:Agneta Willeken, Hamburg: Brief von 1535\nabbr_ddd:Agneta Willeken\ntext-type:Privatbrief\nreference:Hs.: Blatt (r/v)\nreference_secondary:---\nedition:a) Fischer, Christian  (1988): Untersuchungen zur Stadtsprache Hamburgs im 16. Jahrhundert. Abschrift eines Liebesbriefes von Agneta Willeken an Marx Meyer. Wiss. Hausarbeit zur Erlangung des akad. Grades eines Magister Artium. Hamburg: Universität Hamburg.; b) Meier, Jürgen/ Möhn, Dieter (2008): Spuren der Vergangenheit für die Gegenwart. Hundert niederdeutsche Texte zwischen dem 9. und 17. Jahrhundert. Bremen: Verlag Schuster Leer (Schriften des INS).; c) Reincke, Heinrich (1928): Agneta Willeken. Ein Lebensbild aus Wullenwevers Tagen. Lübeck (Pfingstblätter des Hansischen Geschichtsvereins; Blatt 19)\nliterature:Schampier, Andrea (2011): Agneta Willeken: Ein historischer Roman aus Hamburg und Lübeck. 2. Aufl. Fischerhude: Verl. Atelier im Bauernhaus - darin enthalten: Teilübersetzung.\nlibrary:Hamburg: Staatsarchiv Hamburg\nlibrary-shelfmark:Nr. 211-2; Reichskammergericht, W30 I\nonline:---\nscribe_or_printer:unbekannt\nplace:Hamburg\nextent:7 Bl.\ne